In [29]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [30]:
df = pd.read_csv("breast-cancer.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [31]:
print(f"The total number of Columns is {df.shape[1]} and the total number of Rows is {df.shape[0]}.")

The total number of Columns is 32 and the total number of Rows is 569.


In [32]:
df.drop(columns=['id'],inplace=True)

In [33]:
print(f"The total number of Columns is {df.shape[1]} and the total number of Rows is {df.shape[0]}.")

The total number of Columns is 31 and the total number of Rows is 569.


In [34]:
df['diagnosis'].unique()

array(['M', 'B'], dtype=object)

In [35]:
x = df.iloc[:,1:] #All Colum Excepted 'diagnosis'
y = df.iloc[:,0] # 'diagnosis' Column

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [36]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [37]:
lb = LabelEncoder()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [38]:
#Conver the all numpy Arrays to the Tensors.
x_train_tensor = torch.from_numpy(x_train)
x_test_tensor = torch.from_numpy(x_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [39]:
x_test_tensor.shape

torch.Size([114, 30])

In [52]:
class MyfirstNN():
    def __init__(self,x):
        self.weights = torch.rand(x.shape[1],dtype=torch.float64,requires_grad=True)
        self.bias = torch.zeros(1,dtype=torch.float64,requires_grad=True)   

    def forward(self,x):
        z = torch.matmul(x,self.weights) + self.bias
        y_predict = torch.sigmoid(z)
        return y_predict

    def loss(self,y_pred,y):
        #Clamp Predictions to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred,epsilon,1-epsilon)

        #calculate loss
        loss = -(y_train_tensor*torch.log(y_pred) + (1 - y_train_tensor) * torch.log(1 - y_pred)).mean()
        return loss

In [53]:
learning_rate = 0.1
epochs = 25

In [55]:
#Create Model
model = MyfirstNN(x_train_tensor)

#Define Loop
for i in range(epochs): 
    
    #Forward Pass 
    y_predict = model.forward(x_train_tensor)
    print(y_predict)
    
    #Loss Calculation
    loss = model.loss(y_predict,y_train_tensor)
    
    #Backward Pass
    loss.backward()
    
    #Parameters Updates.
    with torch.np_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad

    #zero gradients
    model.weights.grad.zero_
    model.bias.grad.zero_

    #Print Loss in each epoch
    print(f"Epoch: {epoch + 1}, {loss.item()}.")

tensor([9.9997e-01, 9.0583e-03, 3.5375e-02, 6.7524e-04, 9.0905e-01, 9.6725e-03,
        1.8917e-01, 9.9637e-01, 9.7821e-01, 9.9949e-01, 8.5608e-01, 6.4305e-03,
        5.6828e-04, 8.6947e-01, 5.9380e-02, 2.5560e-03, 1.2933e-03, 9.7823e-01,
        1.0000e+00, 9.9031e-01, 3.6948e-05, 2.2563e-02, 3.8354e-01, 1.4836e-01,
        7.4174e-01, 7.9165e-01, 2.4941e-02, 1.0000e+00, 2.4039e-02, 3.9364e-02,
        3.4873e-03, 9.9856e-01, 1.0000e+00, 9.9831e-01, 1.5817e-02, 1.0000e+00,
        1.0000e+00, 9.6191e-01, 1.6614e-01, 5.5964e-05, 9.9955e-01, 6.0037e-03,
        9.9778e-01, 4.5624e-03, 2.8360e-01, 1.0094e-03, 9.9871e-01, 1.0000e+00,
        7.4081e-01, 8.0751e-01, 5.8564e-02, 3.1559e-03, 2.6036e-02, 2.4683e-04,
        9.9993e-01, 2.4798e-03, 1.1580e-03, 7.4564e-01, 1.6639e-02, 3.0663e-03,
        9.6640e-04, 9.6739e-01, 8.8743e-01, 1.0963e-03, 5.6890e-05, 9.0546e-01,
        9.8674e-01, 1.0000e+00, 1.0000e+00, 9.9998e-01, 9.9992e-01, 9.2220e-03,
        8.2147e-01, 6.0203e-01, 9.9998e-

AttributeError: module 'torch' has no attribute 'np_grad'